In [1]:
import pandas as pd
import yfinance as yf
from datetime import date
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os

# List of ticker symbols
list1 = ['0003.HK','0570.HK','0598.HK','0700.HK','0857.HK','0939.HK','0941.HK',
                  '0968.HK','1530.HK','1810.HK','3988.HK','4502.T','5401.T','6098.T',
                  '6367.T','6501.T','6758.T','7203.T','8053.T','8306.T','8604.T',
                  'ADS.GE','AIR.PA', 'ALV.DE','AZN.L','BARC.L','BAYN.BE','BP.L','CCL.L',
                  'MC.PA','NXT.L','RNO.PA','RIO.L','SAP.DE','UBI.PA','ULVR.L','VOW3.DE',
                  'AAL','AAPL','AMD','AMZN','BA','CVX','DAL','DIS','EA','F','GE','GOOG',
                  'HD','INTC','JPM','KO','LDOS','LMT','MCD','META','MRK','MSFT','NFLX',
                  'NKE','NVDA','PFE','PG','RTX','TSLA','V','WMT','XOM']  # Add more as needed

list2 = ['0003.HK','0598.HK','1810.HK','5401.T','6098.T','6367.T','7203.T','8053.T',
                  'AIR.PA', 'ALV.DE','BARC.L','BP.L','MC.PA','NXT.L','SAP.DE','ULVR.L',
                  'AAPL','AMD','BA','GE','JPM','KO','LDOS','LMT','MRK','PFE','PG','RTX','WMT','XOM']

list_ETF = ['1320.T','1618.T','1621.T','1626.T','1631.T',
            'CGW','DIA','EEM','EWA','EWH','EWQ','EWS','EWW','EWZ','FINE','GOAU','ICLN','KBE','MILN',
            'MXI','PAVE','PEJ','QQQE','RNRG','SOXX','SPY','SPYV','VNQ','XHB','XLC','XLE','XLK','XLP',
            'XLU','XLV','XSD']

ticker_symbols = list_ETF

# DataFrame to collect all results
all_results = pd.DataFrame(columns=['Ticker', 'Golden Crosses', 'Death Crosses'])

today = date.today()
date_str = today.strftime('%Y-%m-%d')  # Formats the date as YYYY-MM-DD

# PDF and CSV file paths with today's date
pdf_path = f'ETF_stock_graphs_{date_str}.pdf'
csv_path = f'ETF_stock_analysis_results_{date_str}.csv'

with PdfPages(pdf_path) as pdf:
    for ticker_symbol in ticker_symbols:
        ticker_data = yf.Ticker(ticker_symbol)
        data = ticker_data.history(period='1d', start='2014-1-1', end=today)

        def calculate_moving_average(data, period):
            data[f"Moving_{period}"] = data["Close"].rolling(window=period, min_periods=period).mean()
            return data

        def check_golden_cross(data):
            golden_crosses = []
            death_crosses = []
            for i in range(1, len(data)):
                if not pd.isna(data["Moving_50"].iloc[i]) and not pd.isna(data["Moving_200"].iloc[i]):
                    if data["Moving_50"].iloc[i] >= data["Moving_200"].iloc[i] and data["Moving_50"].iloc[i-1] < data["Moving_200"].iloc[i-1]:
                        golden_crosses.append(data.index[i])
                    elif data["Moving_50"].iloc[i] <= data["Moving_200"].iloc[i] and data["Moving_50"].iloc[i-1] > data["Moving_200"].iloc[i-1]:
                        death_crosses.append(data.index[i])
            return golden_crosses, death_crosses

        data = calculate_moving_average(data, 50)
        data = calculate_moving_average(data, 200)
        golden_crosses, death_crosses = check_golden_cross(data)

        # Collecting results
        result = pd.DataFrame({
            'Ticker': [ticker_symbol],
            'Golden Crosses': [golden_crosses],
            'Death Crosses': [death_crosses]
        })
        all_results = pd.concat([all_results, result], ignore_index=True)

        # Plotting
        plt.figure(figsize=(14, 7))
        plt.plot(data['Close'], label='Close Price')
        plt.plot(data['Moving_50'], label='50-Day MA')
        plt.plot(data['Moving_200'], label='200-Day MA')

        if golden_crosses:
            plt.scatter(golden_crosses, data.loc[golden_crosses, 'Moving_50'], color='gold', label='Golden Cross', marker='^', s=100)
        if death_crosses:
            plt.scatter(death_crosses, data.loc[death_crosses, 'Moving_50'], color='red', label='Death Cross', marker='v', s=100)

        plt.title(f'Stock Price and Moving Averages for {ticker_symbol}')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        
        # Save the plot to PDF
        pdf.savefig()
        plt.close()

# Save results to CSV
all_results.to_csv(csv_path, index=False)

print(f"Analysis complete. Results saved to '{csv_path}'. Graphs are saved in '{pdf_path}'.")


Analysis complete. Results saved to 'ETF_stock_analysis_results_2024-09-25.csv'. Graphs are saved in 'ETF_stock_graphs_2024-09-25.pdf'.
